# Algorithms used

In [62]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import GroupShuffleSplit

In [3]:
def train_test_split(data, sim = True):
    """
    Initialize a train and test split based on search sessions
    Test set consists of 10% of the total data
    Returns all necessary arrays for the EM algorithm
    """
    gss = GroupShuffleSplit(n_splits=1, test_size=0.01, random_state=42)
    
    # Split data indices based on search sessions
    if sim:
        train_idx, test_idx = next(gss.split(data, groups=data.index))
    else:
        train_idx, test_idx = next(gss.split(data, groups=data['session']))
        
    train = data.iloc[train_idx].copy()
    test = data.iloc[test_idx].copy()
    
    train['qd_id'] = train['qd_id'].cat.remove_unused_categories()
    test['qd_id'] = test['qd_id'].cat.remove_unused_categories()
    
    clicks = train['click'].astype(float).to_numpy()
    ranks = train['rank'].astype(int).to_numpy()
    qd_ids = train['qd_id'].cat.codes.astype(int).to_numpy()
    
    clicks_test = test['click'].astype(float).to_numpy()
    ranks_test = test['rank'].astype(int).to_numpy()
    qd_ids_test = test['qd_id'].cat.codes.astype(int).to_numpy()
    
    return clicks, ranks, qd_ids, clicks_test, ranks_test, qd_ids_test

In [12]:
true_bias = np.array([0.7, 0.6, 0.5, 0.4, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05])

In [13]:
def diff(v1, v2):
    """
    Calculates euclidean distance to the true position bias estimates
    """
    return np.sqrt(np.sum(
        (v1 / v1[0] - v2 / v2[0]) ** 2
    ))

In [14]:
def log_likelihood(clicks, E, R):
    """
    Calculates the log likelihood for the EM algorithm
    """
    
    click_probs = np.where(clicks == 1, E * R, 1 - (E * R))
    click_probs[click_probs == 0] = np.finfo(float).eps  # to deal with zeroes
    
    return np.log(click_probs + 0.001).sum() / len(click_probs)

In [43]:
def E_step(bias, ranks, relevance, qd_ids, llambda):
    """
    Calculates the Expectation-step for the EM algorithm
    """
    
    nonp = 1 - (bias[ranks] * relevance[qd_ids])
    nonp += llambda
    E = (bias[ranks] * (1 - relevance[qd_ids])) / nonp
    R = ((1 - bias[ranks]) * relevance[qd_ids]) / nonp
    
    return E, R

In [52]:
def M_step(clicks, ranks, ranks_cnt, qd_ids, qd_cnt, E, R):
    """
    Calculates the Maximization-step for the EM algorithm
    """
    
    bias = np.bincount(ranks, weights=clicks + (1 - clicks) * E) / ranks_cnt
    relevance = np.bincount(qd_ids, weights=clicks + (1 - clicks) * R ) / qd_cnt
    
    return bias, relevance

In [195]:
def EM(data, alpha = 0, beta = 0, learning_rate = 0.1, llambda = 0.1, max_iter = 1000, tol = 1e-6):
    """
    Conducts the Expectation-Maximization (EM) Algorithm
    When alpha/beta is set to zero it conducts the regular EM
    """
     
    # Create train-test split 
    clicks, ranks, qd_ids, clicks_test, ranks_test, qd_ids_test = train_test_split(data) 
    
    ranks_cnt = np.bincount(ranks) 
    bias = np.bincount(ranks, weights=clicks) / ranks_cnt
    qd_cnt = np.bincount(qd_ids)
    relevance = np.bincount(qd_ids, weights=clicks) / qd_cnt
    
    baseline = bias.copy()
    
    loglikelihoods = []
    loglikelihoods_test = []
    distances = []

    for itt in range(max_iter):
        distances.append(diff(bias, true_bias))
        bias_old, relevance_old = bias.copy(), relevance.copy()
        
        # Expectation step for train and test set
        E, R = E_step(bias, ranks, relevance, qd_ids, llambda)      
        E_test, R_test = E_step(bias, ranks_test, relevance, qd_ids_test, llambda)
        
        # Log Likelihood for train and test set
        loglikelihoods.append(log_likelihood(clicks, E, R))
        loglikelihoods_test.append(log_likelihood(clicks_test, E_test, R_test))        
        
        # Maximization step based on only train set
        bias_new, relevance_new = M_step(clicks, ranks, ranks_cnt, qd_ids, qd_cnt, E, R)
        
        # Bounds EM by priors, if a = b = 0 then no priors are used
        if alpha != 0 & beta != 0:
            relevance_new = (alpha +  np.round(qd_cnt * relevance_new)) / (alpha + beta + qd_cnt)
        
        # Updates parameters with learning rate
        bias += (bias_new - bias) * learning_rate
        relevance += (relevance_new - relevance) * learning_rate
        
        # Convergence criteria
        if np.max(np.abs(bias - bias_old)) < tol and np.max(np.abs(relevance - relevance_old)) < tol:
            break
            
    return bias, relevance, baseline, distances, loglikelihoods, loglikelihoods_test

In [36]:
%store EM

Proper storage of interactively declared classes (or instances
of those classes) is not possible! Only instances
of classes in real modules on file system can be %store'd.



In [35]:
%run Simulations.ipynb

Note: you may need to restart the kernel to use updated packages.
Dense Data, Rank-Relevance correlation:  ---------------------------------------------------------------  --------------
Amount of query-document pairs that occurred only once              0.0849979
Amount of query-document pairs that occurred 2-10 times:            0.488738
Amount of query-document pairs that occurred more than 10 times    99.4263
Highest occurence count:                                         5369
Amount of rows:                                                     3.08152e+06
Amount of query-document pairs:                                  4706
---------------------------------------------------------------  --------------
Dense Data, No Rank-Relevance correlation:  ---------------------------------------------------------------  --------------
Amount of query-document pairs that occurred only once              1.00032
Amount of query-document pairs that occurred 2-10 times:            1.80703
Amount 

In [180]:
# Regular EM on dense data scenarios
results_dense_corr = EM(dense_data_corr) # rank-relevance correlation
results_dense_random = EM(dense_data_random) # no rank-relevance correlation

Convergence Reached
Convergence Reached


In [205]:
# Report
print("Dense Simulation,  R-R correlation \n", "Iterations: ", len(results_dense_corr[4]), "\n Euclidean Distance: ", results_dense_corr[3][-1], "\n LL train set :", results_dense_corr[4][-1],  "\n LL test set: ", results_dense_corr[5][-1])
print("\n Dense Simulation, No R-R correlation \n", "Iterations: ", len(results_dense_random[4]), "\n Euclidean Distance: ", results_dense_random[3][-1], "\n LL train set :", results_dense_random[4][-1],  "\n LL test set: ", results_dense_random[5][-1])

Dense Simulation R-R correlation 
 Iterations:  442 
 Euclidean Distance:  0.09705657750994029 
 LL train set : -0.5505569159282709 
 LL test set:  -0.6907556603361568

 Dense Simulation No R-R correlation 
 Iterations:  379 
 Euclidean Distance:  0.15127964014740455 
 LL train set : -0.6021822896001325 
 LL test set:  -0.6267978198923799


In [192]:
# Sparse Data Scenarios

# Runs different priors and regular EM
alphas = [0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9]
betas = [0, 1, 9, 8, 7, 6, 5, 4, 3, 2, 1]

uniform_results = [EM(sparse_data_uniform, alpha = alpha, beta = beta) for alpha, beta in zip(alphas, betas)]
peaked_results = [EM(sparse_data_peaked, alpha = alpha, beta = beta) for alpha, beta in zip(alphas, betas)]

alphas[0], betas[0] = "Regular EM", "Regular EM"

In [206]:
dist = [l[3][-1] for l in uniform_results]
lls = [l[4][-1] for l in uniform_results]
lls_test = [l[5][-1] for l in uniform_results]

print("\n Sparse Simulation, Uniform Relevance \n")
df1 = pd.DataFrame({'Alpha': alphas, 'Beta': betas, 'Euclidean Distances': dist, 'Log Likelihoods Train Set': lls, 'Log Likelihoods Test Set': lls_test})

print(tabulate(df1, headers = 'keys', tablefmt = 'psql'))


 Sparse Simulation, Uniform Relevance 

+----+------------+------------+-----------------------+-----------------------------+----------------------------+
|    | Alpha      | Beta       |   Euclidean Distances |   Log Likelihoods Train Set |   Log Likelihoods Test Set |
|----+------------+------------+-----------------------+-----------------------------+----------------------------|
|  0 | Regular EM | Regular EM |              0.200077 |                   -0.910362 |                  -1.40578  |
|  1 | 1          | 1          |              0.144884 |                   -0.557437 |                  -0.628729 |
|  2 | 1          | 9          |              0.252598 |                   -0.725014 |                  -0.899237 |
|  3 | 2          | 8          |              0.150735 |                   -0.670582 |                  -0.769979 |
|  4 | 3          | 7          |              0.13725  |                   -0.629909 |                  -0.689405 |
|  5 | 4          | 6          

In [207]:
dist = [l[3][-1] for l in peaked_results]
lls = [l[4][-1] for l in peaked_results]
lls_test = [l[5][-1] for l in peaked_results]
 
print("\n Sparse Simulation, Non-Uniform Relevance \n")

df2 = pd.DataFrame({'Alpha': alphas, 'Beta': betas, 'Euclidean Distances': dist, 'Log Likelihoods Train Set': lls, 'Log Likelihoods Test Set': lls_test})

print(tabulate(df2, headers = 'keys', tablefmt = 'psql'))


 Sparse Simulation, Non-Uniform Relevance 

+----+------------+------------+-----------------------+-----------------------------+----------------------------+
|    | Alpha      | Beta       |   Euclidean Distances |   Log Likelihoods Train Set |   Log Likelihoods Test Set |
|----+------------+------------+-----------------------+-----------------------------+----------------------------|
|  0 | Regular EM | Regular EM |             0.426901  |                   -0.820268 |                  -1.00967  |
|  1 | 1          | 1          |             0.0446825 |                   -0.471212 |                  -0.445864 |
|  2 | 1          | 9          |             0.372091  |                   -0.590077 |                  -0.611385 |
|  3 | 2          | 8          |             0.191578  |                   -0.543125 |                  -0.523697 |
|  4 | 3          | 7          |             0.0586518 |                   -0.513512 |                  -0.473874 |
|  5 | 4          | 6      